In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('covid_toy.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

In [7]:
x_train , x_test , y_train ,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=0)

In [8]:
x_train.shape

(80, 5)

# **Without Col Trans**

In [10]:
# simple imputer to fever column due to missing value in it

si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [11]:
# Ordinal Encoding for cough

oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

x_test_cough = oe.fit_transform(x_train[['cough']])

x_train_cough.shape

(80, 1)

In [18]:
# OneHotEncoding for city and gender

ohe = OneHotEncoder(drop='first',sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])
x_train_gender_city.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [19]:
# Extracting age

x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values
x_train_age.shape

(80, 1)

In [22]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough,),axis=1)

#x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)
x_train_transformed.shape

(80, 7)

# **Using COl Transformer**

In [23]:
from sklearn.compose import ColumnTransformer

In [36]:
transformer = ColumnTransformer( transformers=[
              ('tn1' ,SimpleImputer(),['fever']),
              ('tn2' ,OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
              ('tn3' ,OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [37]:
transformer.fit_transform(x_train).shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [38]:
transformer.transform(x_test).shape

(20, 7)